In [46]:
import numpy as np
import open3d as o3d
from tqdm import tqdm

In [47]:
datas = o3d.io.read_point_cloud('Point Cloud Data\Corner.ply')
xyz = np.asarray(datas.points)

In [ ]:
xyz_cp = xyz.copy()


In [48]:
xyz = xyz[np.lexsort(xyz.T)]

In [49]:
Nlrp = 1000
LRP = np.average(xyz[:Nlrp,2])

In [44]:
Thseeds = 0.1

In [50]:
seed = xyz[xyz[:,2] < Thseeds + LRP ]

In [51]:
def SVD(points):
    # 二维，三维均适用
    # 二维直线，三维平面
    pts = points.copy()
    # 奇异值分解
    c = np.mean(pts, axis=0)
    A = pts - c # shift the points
    A = A.T #3*n
    u, s, vh = np.linalg.svd(A, full_matrices=False, compute_uv=True) # A=u*s*vh
    normal = u[:,-1]

    # 法向量归一化
    nlen = np.sqrt(np.dot(normal,normal))
    normal = normal / nlen
    # normal 是主方向的方向向量 与PCA最小特征值对应的特征向量是垂直关系
    # u 每一列是一个方向
    # s 是对应的特征值
    # c >>> 点的中心
    # normal >>> 拟合的方向向量
    return u,s,c,normal

def estimate_parameters(pts):
        # 最小二乘法估算平面模型
        # 只有三个点时，可以直接计算

        _,_,c,n = SVD(pts)

        params = np.hstack((c.reshape(1,-1),n.reshape(1,-1)))[0,:]
        return params

In [52]:
seed_cp = seed
for i in range(10):    
    params = estimate_parameters(pts=seed_cp)
    h = ((xyz[:,0]-params[0]) * params[3] + (xyz[:,1]-params[1])* params[4]+(xyz[:,2]-params[2])* params[5])/(np.sqrt(params[3]**2+params[4]**2+params[5]**2))
    seed_cp = xyz[h < 0.15]

In [53]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(seed_cp)
pcd.paint_uniform_color([1,0,0])
o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 不支持请求的转换操作。 


In [54]:
o3d.visualization.draw_geometries([datas])